In [1]:
import numpy as np
import cv2
from skimage.registration import optical_flow_tvl1
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
import os
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
from multiprocess import Pool
from tqdm import tqdm




### Function

In [14]:
def perform_PIV(video_path, gridSize=30, smooth=True, sigma=30, velScale=1.6):
    print(f"Processing video: {video_path}")
    
    # Directly use video_path instead of vidname
    vid = cv2.VideoCapture(video_path)
    velocity_data = {}
    ret, im1 = vid.read()
    yrange, xrange = im1.shape[:2]
    Ttot = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    delta_T = 1
    tseries = range(0, Ttot - 1)

    # Define meshgrid
    x = np.arange(gridSize/2, xrange - gridSize/2 + 1, gridSize)
    y = np.arange(gridSize/2, yrange - gridSize/2 + 1, gridSize)
    X1, Y1 = np.meshgrid(x, y)
    
    # Get the base directory of the video
    base_directory = os.path.dirname(video_path)
    video_name = os.path.basename(video_path).split('.')[0] + "_new"
    
    # Create the output directory in the same location as the input video
    output_folder = os.path.join(base_directory, video_name)
    print(f"Output folder: {output_folder}")
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Initial frame read
    if im1.ndim == 3:
        im1_gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    else:
        im1_gray = im1
    
    # Add tqdm for the loop to show the progress bar
    for tt in tqdm(tseries[:-1]):
        # Read next frame
        ret, im2 = vid.read()
        if not ret:
            print(f"Couldn't read frame at time {tt}. Skipping...")
            continue

        if im2.ndim == 3:
            im2_gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        else:
            im2_gray = im2

        # Compute the optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(im1_gray, im2_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        VX = flow[..., 0]
        VY = flow[..., 1]

        if smooth:
            VX = cv2.GaussianBlur(VX, (0, 0), sigma)
            VY = cv2.GaussianBlur(VY, (0, 0), sigma)

        UX = cv2.resize(VX, (X1.shape[1], X1.shape[0])) * velScale
        UY = cv2.resize(VY, (X1.shape[1], X1.shape[0])) * velScale

        # Store the velocity vectors in the dictionary
        velocity_data[tt] = (UX, UY)

        # Plot velocity vectors without showing them
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(im1_gray, cmap='gray')

        # Adjust arrow appearance
        arrow_width = 0.002  # Adjust for desired arrow width
        quiver_scale = 350
        head_width = 3
        head_length = 5
        quiver_plot = ax.quiver(X1, Y1, UX, UY, color='c', scale=quiver_scale, width=arrow_width, headwidth=head_width, headlength=head_length)

        plt.axis('off')
        plt.savefig(os.path.join(output_folder, f"velocity_{tt}.png"), bbox_inches='tight', pad_inches=0, dpi=300)
        plt.close(fig)
        im1_gray = im2_gray

    vid.release()

    # After the loop, save the velocity data to a pickle file
    with open(os.path.join(output_folder, 'velocity_data.pkl'), 'wb') as f:
        pickle.dump(velocity_data, f)


In [17]:
def process_video(avi_file):
    full_path = os.path.join(video_directory, avi_file)
    perform_PIV(full_path, gridSize=30)



In [19]:
# Directory containing your .avi videos
video_directory = "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/"

# Get a list of all .avi files in the specified directory
avi_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.endswith('.avi')]

# Process each video without multiprocessing
for video_path in avi_files:
    perform_PIV(video_path, gridSize=30)


Processing video: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/150_MT_1uL_correlation.avi
Output folder: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/150_MT_1uL_correlation_new


  0%|          | 0/243 [00:00<?, ?it/s]

100%|██████████| 243/243 [11:52<00:00,  2.93s/it]


Processing video: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/205_MT_correlation_45_1ul.avi
Output folder: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/205_MT_correlation_45_1ul_new


100%|██████████| 357/357 [17:15<00:00,  2.90s/it]


...

100%|██████████| 357/357 [17:37<00:00,  2.96s/it]


Processing video: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/235_MT_correlation_15ul.avi
Output folder: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/235_MT_correlation_15ul_new


100%|██████████| 357/357 [18:03<00:00,  3.04s/it]


Processing video: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/5_MT_1uL_correlation.avi
Output folder: /Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/correlation_good/5_MT_1uL_correlation_new


100%|██████████| 178/178 [08:44<00:00,  2.95s/it]
